In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("C:\\Users\\lakshay garg\\Documents\\NLP\\shop-product.csv")
df

,shopId,shopName,productsDescription
0,256,XS Stationers,XS stationers at Kailash colony is now online....
1,257,A2Z Chemist,A2Z your friendly Kailash colony chemist is no...
2,260,Vypzee,Vypzee - Taking your trusted local markets onl...
3,268,Durga Fruit Store,Spl. In: Indian and Imported Fruits Suppliers ...
4,269,Pooja Malhotra,Embracing the key elements of romanticism and ...
...,...,...,...
7684,8765,Net Point,All Mobile Accessories - Adhar card PVC Print
7685,8766,Kishan Lal Sabji Supplier,Wholesale
7686,8767,Simran Housekeeping Products,All Housekeeping Products
7687,8768,Indu fashions,deals in: Designer sarees and Lehenga chunni.


In [3]:
df['productsDescription'] = df['shopName'] + ' - ' + df['productsDescription']
df.head()

,shopId,shopName,productsDescription
0,256,XS Stationers,XS Stationers - XS stationers at Kailash colon...
1,257,A2Z Chemist,A2Z Chemist - A2Z your friendly Kailash colony...
2,260,Vypzee,Vypzee - Vypzee - Taking your trusted local ma...
3,268,Durga Fruit Store,Durga Fruit Store - Spl. In: Indian and Import...
4,269,Pooja Malhotra,Pooja Malhotra - Embracing the key elements of...


In [4]:
import spacy
import re
import string
from spacy.lang.en.stop_words import STOP_WORDS


# Load spaCy's English model
spacy_nlp = spacy.load('en_core_web_sm')

# Function for tokenizing abbreviations and words
def spacy_tokenizer(sentence):
    # Check if the input is a string
    if not isinstance(sentence, str):
        return []  # Return an empty list for non-string inputs
    
    # Lowercase the input
    sentence = sentence.lower()

    # Preserve abbreviations (e.g., "n.c.e.r.t" -> "ncert")
    abbreviations = re.findall(r'\b(?:[a-z]\.)+[a-z]\b', sentence)
    sentence = re.sub(r'\b(?:[a-z]\.)+[a-z]\b', lambda x: x.group(0).replace('.', ''), sentence)

    # Remove periods between letters but keep the concatenated version as well
    sentence = re.sub(r'(?<=\w)\.(?=\w)', '', sentence)

    # Tokenize using spaCy
    tokens = spacy_nlp(sentence)

    all_tokens = []

    # Extract individual characters and full tokens
    for token in tokens:
        if token.text.isalpha():
            # Add individual characters as separate tokens
            all_tokens.extend(list(token.text))
            # Add the full word
            all_tokens.append(token.text)

    # Add abbreviations back to the token list
    all_tokens.extend(abbreviations)    
    #lower, strip and lemmatize
    all_tokens = spacy_nlp(sentence)
    
    all_tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in all_tokens]

    # Return the token list
    return list(set(all_tokens))  # Removing duplicates if needed

# Example usage with a float value
print(spacy_tokenizer('N.C.E.R.T is a book for cbse: used by all students'))

['student', 'ncert', 'a', 'use', 'by', 'all', ':', 'book', 'be', 'for', 'cbse']


In [5]:
print ('Cleaning and Tokenizing...')
%time df['Description_tokenized'] = df['productsDescription'].map(lambda x: spacy_tokenizer(x))

Df_token = df['Description_tokenized']

df.head()

Cleaning and Tokenizing...
CPU times: total: 1min 40s
Wall time: 2min 20s


,shopId,shopName,productsDescription,Description_tokenized
0,256,XS Stationers,XS Stationers - XS stationers at Kailash colon...,"[and, sheaffer, online, xs, colony, :, station..."
1,257,A2Z Chemist,A2Z Chemist - A2Z your friendly Kailash colony...,"[friendly, chemist, and, online, colony, ayurv..."
2,260,Vypzee,Vypzee - Vypzee - Taking your trusted local ma...,"[flat, to, and, local, team, online, colony, y..."
3,268,Durga Fruit Store,Durga Fruit Store - Spl. In: Indian and Import...,"[, delivery, melon-, and, basket, :, passion, ..."
4,269,Pooja Malhotra,Pooja Malhotra - Embracing the key elements of...,"[and, to, contemporary, fashion-, traditional,..."


In [6]:
import gensim
from gensim import corpora

#creating term dictionary
%time dictionary = corpora.Dictionary(Df_token )

#filter out terms which occurs in less than 4 documents and more than 20% of the documents.
#NOTE: Since we have smaller dataset, we will keep this commented for now.

#dictionary.filter_extremes(no_below=1, no_above=1)
#from gensim.parsing.preprocessing import STOPWORDS
#stoplist = STOPWORDS


#list of few which which can be further removed
stoplist = set('hello and if this can would should could tell ask stop come go is we deal in is at you get it a be')
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)

CPU times: total: 93.8 ms
Wall time: 152 ms


In [7]:
#corpus = [dictionary.doc2bow(desc) for desc in Df_token]
#word_frequencies = [[(dictionary[id], frequency) for id, frequency in line] for line in corpus[0:len(corpus)]]
# Bag-of-Words transformation for each document in Df_token
corpus = [dictionary.doc2bow(desc) for desc in Df_token]

# Function to convert the bag-of-words corpus into human-readable word frequency format
def get_word_frequencies(corpus, dictionary, n=len(corpus)):
    word_frequencies = []
    for line in corpus[:n]:  # Limit to first `n` documents
        doc_frequencies = [(dictionary[id], freq) for id, freq in line]
        word_frequencies.append(doc_frequencies)
    return word_frequencies

# Call the function for the first 3 documents
word_frequencies = get_word_frequencies(corpus, dictionary, n=len(corpus))

# Display the word frequencies for the first 3 documents
for doc_id, doc in enumerate(word_frequencies):
    print(f"Document {doc_id + 1}:")
    for word, freq in doc:
        print(f"{word}: {freq}")
    print("\n")


Document 1:
-: 1
.: 1
:: 1
and: 1
art: 1
at: 1
be: 1
books-: 1
can: 1
cartridges-: 1
children: 1
colony: 1
cross: 1
deliver: 1
for: 1
general: 1
get: 1
hp: 1
it: 1
kailash: 1
material-: 1
now: 1
office: 1
online: 1
order: 1
pen: 1
printer: 1
sheaffer: 1
stationary-: 1
stationer: 1
stationers: 1
supplies-: 1
xs: 1
you: 1


Document 2:
-: 1
.: 1
and: 1
be: 1
colony: 1
deliver: 1
get: 1
it: 1
kailash: 1
now: 1
online: 1
order: 1
&: 1
a2z: 1
allopathic-: 1
any: 1
ayurvedic-: 1
baby: 1
chemist: 1
cosmetic: 1
friendly: 1
kind: 1
of: 1
product: 1
your: 1


Document 3:
-: 1
.: 1
and: 1
at: 1
be: 1
colony: 1
for: 1
it: 1
kailash: 1
online: 1
order: 1
your: 1
%: 1
1st: 1
30th: 1
5: 1
all: 1
avail: 1
bear: 1
by: 1
cashback: 1
completion: 1
delkailashcolonyvypzeecom: 1
flat: 1
from: 1
local: 1
market: 1
merchant: 1
no: 1
nov: 1
oct: 1
offer: 1
on: 1
or: 1
partner: 1
place: 1
purchase: 1
record: 1
responsibility: 1
shop: 1
take: 1
team: 1
the: 1
to: 1
trust: 1
valid: 1
value: 1
vypzee: 1
will: 1




art: 1
in: 1
designer: 1
deal: 1
suit: 1
etc: 1
lehenga: 1
sarees: 1
jbl: 1


Document 2749:
-: 1
:: 1
merchant: 1
: 1
in: 1
deal: 1
enterprise: 1
wholesale: 1
clothe: 1
rajiv: 1


Document 2750:
-: 1
:: 1
all: 1
: 1
good: 1
in: 1
house: 1
deal: 1
beauty: 1
lucknow: 1
chikan: 1


Document 2751:


Document 2752:
-: 1
:: 1
and: 1
: 1
in: 1
deal: 1
choice: 1
new: 1
gent: 1
boy: 1
sherwani: 1


Document 2753:
-: 1
.: 1
:: 1
and: 1
: 1
in: 1
deal: 1
sherwani-: 1
suit: 1
wedding: 1
lal: 1
etc: 1
jain: 1
lehenga: 1
bros: 1
chunni: 1
hazari: 1


Document 2754:
-: 1
.: 1
:: 1
kind: 1
of: 1
all: 1
: 1
in: 1
deal: 1
footwear: 1
kapoor: 1
mall: 1
pp: 1
dzign: 1


Document 2755:
-: 1
:: 1
kind: 1
of: 1
all: 1
: 1
in: 1
deal: 1
footwear: 1
golden: 1


Document 2756:
-: 1
&: 1
: 1
electrical: 1
wire: 1
cable: 1
ishani: 1


Document 2757:
-: 1
kind: 1
of: 1
all: 1
: 1
home: 1
special: 1
decoration: 1
lite: 1
shri: 1
big: 1
jai: 1
laser: 1
rope: 1
light: 1
smd: 1
dj: 1
dipper: 1


Document 2758:


Docu

Document 6897:
-: 1
&: 1
of: 1
all: 1
: 1
work: 1
plate: 1
handle: 1
manufacturer: 1
engineering: 1
rado: 1
motorcycle: 1
rikshaw: 1
forkleg: 1
scooty: 1
stem: 1


Document 6898:


Document 6899:
-: 1
of: 1
: 1
sethi: 1
spare: 1
importer: 1
part: 1
rickshaw: 1
automotive: 1


Document 6900:


Document 6901:


Document 6902:
-: 1
&: 1
of: 1
all: 1
: 1
material: 1
international: 1
type: 1
sheet: 1
foam: 1
packing: 1
expand: 1
polythylene: 1


Document 6903:
-: 1
&: 1
all: 1
: 1
available: 1
jewellery: 1
real: 1
type: 1
rk: 1
stone: 1
gem: 1
babbar: 1


Document 6904:


Document 6905:


Document 6906:
-: 1
:: 1
&: 1
: 1
in: 1
specialist: 1
tyre: 1
singh: 1
wheel: 1
rickshaw: 1
scooty: 1
tubes: 1


Document 6907:


Document 6908:


Document 6909:
-: 1
&: 1
: 1
wedding: 1
drive: 1
self: 1
event: 1
rent: 1
car: 1
long: 1
term: 1
corporation: 1
hotel: 1
rental: 1
lease: 1
chuffer: 1
rbt: 1
transportation-: 1


Document 6910:
-: 1
&: 1
: 1
food: 1
non: 1
vegetarian: 1
foods: 1
shahi: 1


Docum

In [ ]:
Df_token_tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictionary)
Df_token_lsi_model = gensim.models.LsiModel(Df_token_tfidf_model[corpus], id2word=dictionary, num_topics=12000)

#Serialize and Store the corpus locally for easy retrival whenver required.
gensim.corpora.MmCorpus.serialize('Df_token_tfidf_model_mm', Df_token_tfidf_model[corpus])
gensim.corpora.MmCorpus.serialize('Df_token_lsi_model_mm',Df_token_lsi_model[Df_token_tfidf_model[corpus]])

#Load the indexed corpus
Df_token_tfidf_corpus = gensim.corpora.MmCorpus('Df_token_tfidf_model_mm')
Df_token_lsi_corpus = gensim.corpora.MmCorpus('Df_token_lsi_model_mm')

#Load the MatrixSimilarity
from gensim.similarities import MatrixSimilarity
Df_token_index = MatrixSimilarity(Df_token_lsi_corpus, num_features = Df_token_lsi_corpus.num_terms)

In [ ]:
print(Df_token_lsi_corpus.num_terms)

In [ ]:
from operator import itemgetter
import pandas as pd


def search_product(search_term):
    # Tokenize the query and convert it into a bag-of-words format using the dictionary
    query_bow = dictionary.doc2bow(spacy_tokenizer(search_term))
    
    # Convert the BOW into the tf-idf representation
    query_tfidf = Df_token_tfidf_model[query_bow]
    
    # Convert tf-idf into LSI space
    query_lsi = Df_token_lsi_model[query_tfidf]

    # Set the number of best matches to retrieve
    Df_token_index.num_best = 100

    # Perform similarity search using the LSI model
    Df_token_list = Df_token_index[query_lsi]
    #Df_token_list = Df_token_lsi_corpus[query_lsi]

    # Sort the results by relevance score (2nd item in the tuple)
    Df_token_list.sort(key=itemgetter(1), reverse=True)
    
    Df_token_names = []

    # Iterate over the top results and extract relevant information
    for j, Df_token in enumerate(Df_token_list):
        Df_token_names.append(
            {
                'Relevance': round((Df_token[1] * 100), 2),
                'Company_name':df['shopName'][Df_token[0]],
                #'Product_name': df['ProductName'][Df_token[0]],  # Ensure df_Df_token is a DataFrame with title and plot columns
                'Product_Description': df['productsDescription'][Df_token[0]],
                #'ID':df['ProductID'][Df_token[0]],
            }
        )

        # Break after collecting the top N results (num_best)
        if j == (Df_token_index.num_best - 1):
            break

    # Return the results as a DataFrame
    return pd.DataFrame(Df_token_names, columns=['Relevance','Company_name', 'Product_Description'])


In [ ]:
search_product('electronics')